In [1]:
import sklearn
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from math import isnan
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.cross_validation import cross_val_score
from sklearn import svm

/Users/mahmoodhegazy/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [17]:
training_set = pd.read_csv("train_input.csv")

In [18]:
training_set_array = np.array(training_set) #convert it to numpy array for manipulation

In [19]:
training_set_array

array([[0,
        "<speaker_1> seaworld ceo steps down amid tanking revenues - company announces undisclosed number of layoffs after volley of public criticism for its treatment of killer whales , underscored in <number> documentary blackfish outsideonline . com </s> <speaker_2> nothing like a little propaganda coupled with a good media campaign to sway public perception . pathetic . </s> <speaker_3> are you saying that the status quo should be upheld , and that maybe , just maybe , seaworld shouldn 't make at least a couple of changes regarding the well - being of their animals and safety of the trainers ? ? </s> </d>\n"],
       [1,
        '<speaker_1> strickland chargers owner dean spanos and goldman - sachs agree to build la stadium insidestl . com </s> <speaker_2> utkevinacee twitter . com @chasingsunsets7 @utkevinacee any truth to the spanos goldman sachs deal to a build stadium in la ? no . </s> <speaker_3> right . when i look for facts i definitely look to acee s </s> </d>\n'

In [20]:
training_set_array = np.delete(training_set_array, [0], 1)  # remove headers


In [21]:
training_set_array

array([[ "<speaker_1> seaworld ceo steps down amid tanking revenues - company announces undisclosed number of layoffs after volley of public criticism for its treatment of killer whales , underscored in <number> documentary blackfish outsideonline . com </s> <speaker_2> nothing like a little propaganda coupled with a good media campaign to sway public perception . pathetic . </s> <speaker_3> are you saying that the status quo should be upheld , and that maybe , just maybe , seaworld shouldn 't make at least a couple of changes regarding the well - being of their animals and safety of the trainers ? ? </s> </d>\n"],
       [ '<speaker_1> strickland chargers owner dean spanos and goldman - sachs agree to build la stadium insidestl . com </s> <speaker_2> utkevinacee twitter . com @chasingsunsets7 @utkevinacee any truth to the spanos goldman sachs deal to a build stadium in la ? no . </s> <speaker_3> right . when i look for facts i definitely look to acee s </s> </d>\n'],
       [ "<speake

In [22]:
out_set = pd.read_csv('train_output.csv')

In [23]:
out_set_array = np.array(out_set)

In [24]:
out_set_array = np.delete(out_set_array, [0], 1)

In [25]:
#create a scikit countvectorizer, this will do stemming and remove stop words as well as term frequency
count_vect = CountVectorizer(min_df = 2, lowercase = True, stop_words = 'english', strip_accents = 'unicode') 

In [26]:
training_set_array = training_set_array.ravel()

In [27]:
training_set_array

array([ "<speaker_1> seaworld ceo steps down amid tanking revenues - company announces undisclosed number of layoffs after volley of public criticism for its treatment of killer whales , underscored in <number> documentary blackfish outsideonline . com </s> <speaker_2> nothing like a little propaganda coupled with a good media campaign to sway public perception . pathetic . </s> <speaker_3> are you saying that the status quo should be upheld , and that maybe , just maybe , seaworld shouldn 't make at least a couple of changes regarding the well - being of their animals and safety of the trainers ? ? </s> </d>\n",
       '<speaker_1> strickland chargers owner dean spanos and goldman - sachs agree to build la stadium insidestl . com </s> <speaker_2> utkevinacee twitter . com @chasingsunsets7 @utkevinacee any truth to the spanos goldman sachs deal to a build stadium in la ? no . </s> <speaker_3> right . when i look for facts i definitely look to acee s </s> </d>\n',
       "<speaker_1> in

In [28]:
inputWordCounter = count_vect.fit_transform(training_set_array) # feed the data to the vectorizor

In [30]:
#create term frequency inverse document frequncy vector from our data
term_frequency_transformer = TfidfTransformer(use_idf=True).fit(inputWordCounter) 

In [31]:
term_frequency_inverse_doc = term_frequency_transformer.transform(inputWordCounter)

In [32]:
out_set_array = out_set_array.ravel()

In [33]:
out_set_array

array(['news', 'nfl', 'soccer', ..., 'nba', 'soccer', 'news'], dtype=object)

In [43]:
test_set = pd.read_csv("test_input.csv")
test_set_array = np.array(test_set)
test_set_array = np.delete(test_set_array,[0],1) #remove fucking headers

In [44]:
test_set_array = test_set_array.ravel()

In [45]:
#feed it through the same vectorizing process we did to our training data
#transformed_test_count = count_vect.fit_transform(test_set_array)
test_counter = count_vect.transform(test_set_array) 

In [46]:
test_set_frequency = term_frequency_transformer.transform(test_counter)

In [48]:
model = svm.LinearSVC() #create a linear svm model

In [50]:
model.fit(term_frequency_inverse_doc, out_set_array) #train it using our input and out training data

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [51]:
predictions = model.predict(test_set_frequency)

In [52]:
predictions

array(['politics', 'worldnews', 'worldnews', ..., 'nba', 'hockey', 'movies'], dtype=object)

In [55]:
to_submit = pd.DataFrame([[i,predictions[i]] for i in range(len(predictions))], columns = ["id","predicted category"])

In [56]:
to_submit

,id,predicted category
0,0,politics
1,1,worldnews
2,2,worldnews
3,3,politics
4,4,worldnews
5,5,movies
6,6,movies
7,7,movies
8,8,worldnews
9,9,movies


In [67]:
test_out_set = pd.read_csv("test_predict_random.csv")

In [69]:
y_test = test_out_set.category.tolist()

In [65]:
from sklearn.metrics import classification_report

In [75]:
y_test

['worldnews',
 'nba',
 'worldnews',
 'hockey',
 'news',
 'news',
 'nba',
 'movies',
 'movies',
 'news',
 'nfl',
 'soccer',
 'politics',
 'worldnews',
 'nfl',
 'politics',
 'worldnews',
 'soccer',
 'hockey',
 'nfl',
 'nba',
 'worldnews',
 'politics',
 'politics',
 'soccer',
 'movies',
 'soccer',
 'movies',
 'news',
 'politics',
 'nba',
 'soccer',
 'nba',
 'worldnews',
 'soccer',
 'politics',
 'politics',
 'soccer',
 'news',
 'politics',
 'nfl',
 'worldnews',
 'news',
 'hockey',
 'movies',
 'nfl',
 'movies',
 'soccer',
 'worldnews',
 'movies',
 'nfl',
 'soccer',
 'worldnews',
 'soccer',
 'nfl',
 'nba',
 'soccer',
 'movies',
 'movies',
 'movies',
 'soccer',
 'nba',
 'nfl',
 'soccer',
 'hockey',
 'politics',
 'hockey',
 'nba',
 'movies',
 'worldnews',
 'movies',
 'nba',
 'politics',
 'worldnews',
 'news',
 'nba',
 'nba',
 'hockey',
 'hockey',
 'news',
 'news',
 'politics',
 'hockey',
 'nfl',
 'hockey',
 'nba',
 'worldnews',
 'politics',
 'hockey',
 'soccer',
 'worldnews',
 'nba',
 'nba',
 

In [78]:
predictions = predictions.tolist()

In [81]:
print(classification_report(y_test,predictions))

             precision    recall  f1-score   support

     hockey       0.13      0.13      0.13      6680
     movies       0.13      0.14      0.14      6750
        nba       0.12      0.11      0.11      6440
       news       0.13      0.13      0.13      6704
        nfl       0.12      0.11      0.12      6630
   politics       0.13      0.12      0.12      6637
     soccer       0.13      0.13      0.13      6790
  worldnews       0.12      0.13      0.13      6587

avg / total       0.13      0.13      0.13     53218

